In [10]:
import importlib
import pandas as pd
import numpy as np
import koopomics as ko
import torch
importlib.reload(ko)


<module 'koopomics' from '/Users/daviddornig/Documents/Master_Thesis/Bioinf/Code/philipp-trinh/KOOPOMICS/koopomics/__init__.py'>

In [18]:
kmatrix = torch.zeros(4, 4)

# Get the indices for the upper triangular part (excluding the diagonal)
upper_indices = torch.triu_indices(4, 4, offset=1)

# Assign values to the upper triangular part
# Using the same value (e.g., 4) for demonstration
values = torch.tensor([[4.3, 4.3, 4.1, 4.3, 4.2, 4.1]])  # This needs to match the number of elements in the upper triangle

# Assigning values to the upper triangle of kmatrix
kmatrix[upper_indices[0], upper_indices[1]] = values

# Assign negative values to the lower triangle of kmatrix
kmatrix[upper_indices[1], upper_indices[0]] = -values

In [19]:
kmatrix

tensor([[ 0.0000,  4.3000,  4.3000,  4.1000],
        [-4.3000,  0.0000,  4.3000,  4.2000],
        [-4.3000, -4.3000,  0.0000,  4.1000],
        [-4.1000, -4.2000, -4.1000,  0.0000]])

In [17]:
kmatrix[upper_indices[0], upper_indices[1]]

tensor([0., 0., 0., 0., 0., 0.])

In [2]:
print(dir(ko))


['BandedKoopmanMatrix', 'DataLoader', 'Dataset', 'DiffeomMap', 'F', 'FFLinearizer', 'FF_AE', 'InvKoop', 'Koop', 'KoopmanModel', 'LinearizingKoop', 'SkewSymmetricMatrix', 'TimeSeriesDataset', 'Variable', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'build_nn_functions', 'clear_output', 'collate_fn', 'data_loader', 'data_prep', 'dataloader_AE', 'embeddingANN', 'get_dynamic_targets', 'get_validation_targets', 'koopmanANN', 'lr_scheduler', 'model', 'model_loader', 'nn', 'np', 'optim', 'pd', 'plt', 'rnn_utils', 'test', 'test_embedding', 'test_utils', 'torch', 'train', 'train_embedding', 'train_utils', 'training', 'update_batch_loss_plot', 'update_batch_loss_subplots', 'update_batch_loss_subplots_embedding']


In [3]:
embedding = ko.FF_AE([264,1000,20],[20,1000,264])

linearizer = ko.FFLinearizer([20, 1000, 20], [20, 1000, 20])

koop = ko.InvKoop(latent_dim=20)

operator = ko.LinearizingKoop(linearizer=linearizer,koop = koop)

TestingKoop2 = ko.KoopmanModel(embedding=embedding,
                               operator=operator)

Model loaded with:
FF_AE module
LinearizingKoop module
An invertible Koop


In [7]:
embedding_model = ko.FF_AE([50,2000,30], [30,2000,50])
operator_model = ko.LinearizingKoop(linearizer=ko.FFLinearizer([30,2000,40], [40,2000,30]), koop=ko.InvKoop(latent_dim=40, reg='skewsym'))

TestingKoop = ko.KoopmanModel(embedding=embedding_model, operator=operator_model)


Banded Matrix initialized, with 194 of 1600 matrix elements trainable
Banded Matrix initialized, with 194 of 1600 matrix elements trainable
Model loaded with:
FF_AE module
LinearizingKoop module
An invertible Koop


In [8]:
def count_parameters_detailed(model):
    for name, param in model.named_parameters():
        print(f"{name}: {param.numel()}")
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f'Total trainable parameters: {total_params}')

# Example usage
count_parameters_detailed(TestingKoop)

embedding.encode.0.weight: 100000
embedding.encode.0.bias: 2000
embedding.encode.2.weight: 60000
embedding.encode.2.bias: 30
embedding.decode.0.weight: 60000
embedding.decode.0.bias: 2000
embedding.decode.2.weight: 100000
embedding.decode.2.bias: 50
operator.linearizer.lin_encode.0.weight: 60000
operator.linearizer.lin_encode.0.bias: 2000
operator.linearizer.lin_encode.2.weight: 80000
operator.linearizer.lin_encode.2.bias: 40
operator.linearizer.lin_decode.0.weight: 80000
operator.linearizer.lin_decode.0.bias: 2000
operator.linearizer.lin_decode.2.weight: 60000
operator.linearizer.lin_decode.2.bias: 30
operator.koop.fwd_banded_params: 194
operator.koop.bwd_banded_params: 194
Total trainable parameters: 608538


In [4]:
pregnancy_df = pd.read_csv('/Users/daviddornig/Documents/Master_Thesis/Bioinf/Code/philipp-trinh/KOOPOMICS/input_data/pregnancy/pregnancy_interpolated_normalized.csv')

sample_id = 'Subject ID'
time_id = 'Gestational age (GA)/weeks'

feature_list = pregnancy_df.columns[6:]

train_set_df = pregnancy_df[pregnancy_df['Cohort'] == 'Discovery'].copy()
test_set_df = pregnancy_df[pregnancy_df['Cohort'] == 'Validation (Test Set 1)'].copy()

train_dataloader = ko.dataloader_AE(train_set_df, feature_list, sample_id=sample_id, time_id=time_id, batch_size=5)
test_dataloader = ko.dataloader_AE(test_set_df, feature_list, sample_id=sample_id, time_id=time_id, batch_size=5)


In [5]:
embedding_model = ko.FF_AE([264,100,100,30], [30,100,100,264])
operator_model = ko.LinearizingKoop(linearizer=ko.FFLinearizer([20,100,20], [20,100,20]), koop=ko.InvKoop(latent_dim=20))

TestingKoop = ko.KoopmanModel(embedding=embedding_model, operator=operator_model)


Model loaded with:
FF_AE module
LinearizingKoop module
An invertible Koop


In [6]:
embedding_model = ko.FF_AE([264,100,100,100,100,20], [20,100,100,100,100,264])
operator_model = ko.LinearizingKoop(linearizer=ko.FFLinearizer([20,30,30,30,20], [20,30,30,30,20]), koop=ko.InvKoop(latent_dim=20))

TestingKoop3 = ko.KoopmanModel(embedding=embedding_model, operator=operator_model)


Model loaded with:
FF_AE module
LinearizingKoop module
An invertible Koop


In [7]:
loss_dict = ko.test_embedding(TestingKoop, train_dataloader)

In [8]:
loss_dict

{'test_identity_loss': tensor(1.2211)}

In [9]:
for param in TestingKoop.embedding.parameters():
    print(param)

Parameter containing:
tensor([[-0.0038,  0.0268, -0.0187,  ..., -0.0313,  0.0604, -0.0152],
        [-0.0372,  0.0422, -0.0468,  ...,  0.0378, -0.0596, -0.0056],
        [-0.0129, -0.0458,  0.0193,  ...,  0.0178, -0.0585, -0.0239],
        ...,
        [ 0.0009, -0.0186,  0.0317,  ..., -0.0335, -0.0091,  0.0328],
        [ 0.0285, -0.0533,  0.0405,  ..., -0.0259, -0.0211,  0.0553],
        [ 0.0188,  0.0169,  0.0430,  ...,  0.0202,  0.0408, -0.0126]],
       requires_grad=True)
Parameter containing:
tensor([-0.0494,  0.0431,  0.0455, -0.0136, -0.0540,  0.0125,  0.0077, -0.0015,
        -0.0568, -0.0613, -0.0478,  0.0553,  0.0138,  0.0259,  0.0257, -0.0026,
        -0.0360,  0.0496, -0.0575,  0.0018, -0.0569, -0.0220,  0.0338,  0.0305,
         0.0376,  0.0469, -0.0604,  0.0198,  0.0550, -0.0412,  0.0061,  0.0184,
        -0.0301, -0.0552,  0.0523,  0.0303,  0.0236, -0.0355,  0.0377, -0.0280,
         0.0003,  0.0356, -0.0553,  0.0303,  0.0033,  0.0514,  0.0450,  0.0465,
         0.0505

In [10]:
import torch
input_df = pregnancy_df.copy()
metabolite_names = input_df.columns[6:]
# df must have feature from column 6 onward and metadata until column 6.
#input_df[metabolite_names] =  input_df[metabolite_names].sub(average_row_mean, axis=0)

input_tensor = torch.tensor(input_df.iloc[:, 6:].values, dtype=torch.float32)
input_tensor[0]

tensor([ 0.6695, -1.4424, -2.2023, -0.6105,  0.5704,  1.4733,  1.6917,  0.6347,
         3.4789,  3.4759,  1.1653, -0.6691,  0.4019, -1.0045,  0.9781, -0.6744,
         0.3813,  2.5527, -0.3893, -2.1971,  0.7286,  1.0932, -4.3114, -1.3493,
         1.6866,  1.1539, -1.9949, -3.2171, -1.5585,  3.3270, -3.1401, -4.0659,
        -0.0598, -2.1093, -2.5716, -1.2221, -7.8121,  0.6287, -0.1930,  2.8206,
        -0.5932, -1.5778, -1.4347, -4.4832, -1.8511,  0.2475, -0.6552, -5.0469,
        -4.3929,  1.2227, -0.5278,  4.5114, -0.5930, -3.3687,  4.2844,  3.0976,
         0.2213,  0.6778, -1.4817, -0.8550,  2.6132, -1.0299,  2.7675, -0.7996,
        -2.0322,  1.4367,  1.1702, -0.3100,  2.8633,  0.2742,  2.2207, -1.6458,
         0.5513,  2.5245,  1.9290,  3.4519, -0.1615,  0.9453,  0.3212,  3.4990,
        -2.9123, -2.2305, -2.5669, -0.0283, -2.3486,  1.6477, -2.0056, -1.2415,
         1.2639,  3.4499, -0.7323,  2.7940,  0.9982,  5.6954, -2.1027,  0.7507,
        -0.9040,  1.6283, -1.8136, -0.76

In [11]:
embedded, identity = TestingKoop.embed(input_tensor[0])

In [12]:
identity

tensor([ 9.5092e-02, -3.1404e-02, -8.6806e-02, -7.3072e-02,  6.0250e-02,
         5.3340e-03,  4.5974e-02,  6.6905e-02, -9.1596e-02, -8.3272e-02,
         1.2407e-02,  2.0359e-02,  5.1302e-02, -5.3969e-02, -1.2995e-02,
         7.1834e-02,  1.0019e-01, -4.0232e-02,  2.6695e-02,  1.4535e-02,
         5.6430e-02, -3.4332e-03, -4.0653e-02,  1.5067e-01, -3.5673e-02,
        -7.8587e-03, -5.9709e-02, -3.6854e-02,  4.6105e-02,  7.7306e-02,
        -1.9287e-02, -1.0766e-02,  5.4686e-02, -4.7930e-03,  6.5883e-02,
        -1.5374e-02, -4.9112e-02, -1.9308e-02, -9.3314e-02,  5.1468e-02,
        -6.8226e-02,  7.8697e-02, -1.4859e-01,  4.1931e-02, -8.3885e-02,
        -1.0854e-01, -3.7191e-02, -1.1196e-01, -6.6694e-02,  8.2445e-02,
         4.8848e-02,  7.1210e-02,  7.5692e-02, -4.4863e-02,  1.0217e-02,
        -1.4807e-02, -5.0222e-02,  1.4105e-02,  5.8454e-02,  9.9736e-02,
        -4.6455e-02, -1.2511e-02,  5.0212e-02, -6.2435e-02,  9.1552e-02,
        -2.8149e-02, -7.2084e-02,  3.7581e-02,  1.0

In [13]:

# Run training loop
ko.train_embedding(TestingKoop2, train_dataloader, test_dataloader, lr= 0.001, learning_rate_change=0.99, num_epochs=200, weight_decay=0.001, gradclip=5, model_name='TestingKoop')


----------Training epoch--------
----------------0---------------

Training interrupted. Saving model...


In [14]:
import torch
torch.save(TestingKoop.state_dict(), 'TestingKoop3.pth')

In [15]:
import torch
import torch.nn.utils.rnn as rnn_utils

# Create some sample variable-length sequences (tensors)
# For example, we have 3 sequences with different lengths
sequence_1 = torch.tensor([[1,1], [2,2], [3,3]])  # Shape: (3, 1)
sequence_2 = torch.tensor([[4,4], [5,5]])       # Shape: (2, 1)
sequence_3 = torch.tensor([[6,6], [1,7], [1,8], [9,1]])  # Shape: (4, 1)

# Put them in a list
sequences = [sequence_1, sequence_2, sequence_3]

# Pad sequences to the maximum length using pad_sequence
padded_sequences = rnn_utils.pad_sequence(sequences, batch_first=True)

# Output the padded sequences
print("Padded Sequences:\n", padded_sequences)
print("\nPadded Shape:", padded_sequences.shape)  # Should be (batch_size, max_length, num_features)

# Display original lengths for reference
lengths = torch.tensor([len(seq) for seq in sequences])
print("\nOriginal Lengths:", lengths)  # Original lengths of each sequence
mask = (padded_sequences != 0).float()
print(mask)

Padded Sequences:
 tensor([[[1, 1],
         [2, 2],
         [3, 3],
         [0, 0]],

        [[4, 4],
         [5, 5],
         [0, 0],
         [0, 0]],

        [[6, 6],
         [1, 7],
         [1, 8],
         [9, 1]]])

Padded Shape: torch.Size([3, 4, 2])

Original Lengths: tensor([3, 2, 4])
tensor([[[1., 1.],
         [1., 1.],
         [1., 1.],
         [0., 0.]],

        [[1., 1.],
         [1., 1.],
         [0., 0.],
         [0., 0.]],

        [[1., 1.],
         [1., 1.],
         [1., 1.],
         [1., 1.]]])


In [16]:
plot_df_4 = pregnancy_df.copy()
plot_df_3 = pd.read_csv('/Users/daviddornig/Documents/Master_Thesis/Bioinf/Code/philipp-trinh/KOOPOMICS/input_data/pregnancy/pregnancy_interpolated_preprocessed.csv')
time_identifier = time_id
average_row_mean = 11.51776370565249

In [17]:
input_df

,Subject ID,Cohort,Labor onset,Gestational age (GA)/days,Gestational age (GA)/weeks,Birth GA/weeks,(+)-.alpha.-Tocopherol,(R)-2-Hydroxycaprylic acid,"1,4-Dihydroxybenzene",1-Methylxanthine,...,"cis-5,8,11,14,17-Eicosapentaenoic acid",cis-5-Dodecenoic acid,cis-9-Palmitoleic acid,d-LIMONENE,gamma-CEHC,gamma-Glutamylleucine,ketoisocaproic acid,p-Cresol,phenylacetylglutamine,trans-Vaccenic acid
0,DP01,Discovery,natural,35,5,38.714286,0.669483,-1.442366,-2.202257,-0.610471,...,-0.334473,-0.718353,0.541719,-5.874631,-1.021789,0.640099,2.859923,-0.882611,2.745482,-0.367721
1,DP01,Discovery,natural,42,6,38.714286,1.126102,-1.646382,-3.993826,-0.562819,...,-0.499299,-1.287621,0.326793,-6.020122,-1.669941,0.592381,2.700036,-0.906877,2.484140,-0.593080
2,DP01,Discovery,natural,49,7,38.714286,2.115120,-1.492003,-7.086519,-1.801814,...,-0.980554,-2.191199,0.098945,-4.780913,-2.175070,0.656504,2.707352,-0.669747,2.660712,-0.812489
3,DP01,Discovery,natural,56,8,38.714286,0.608111,-1.264217,-6.266998,-1.712359,...,-0.845571,-2.727090,0.356457,-4.321975,-1.495426,1.087006,3.231475,-0.946232,2.524433,-0.362185
4,DP01,Discovery,natural,63,9,38.714286,1.139878,-1.355143,-3.387730,-1.168318,...,-0.729213,-2.544836,0.313885,-4.938126,-1.485728,0.811594,3.421370,-1.213799,2.358499,-0.432032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,DP30,Validation (Test Set 1),natural,245,35,38.857143,0.597496,-0.480410,-4.976716,-1.282269,...,-0.464712,-1.987350,0.582346,-4.883724,-2.164800,0.754066,2.848290,-0.712530,3.369940,0.033240
697,DP30,Validation (Test Set 1),natural,252,36,38.857143,0.951263,0.025362,-4.614178,-1.175928,...,-0.469523,-2.080101,0.603483,-4.793372,-2.382641,0.367110,2.954354,-0.650140,3.157251,-0.612668
698,DP30,Validation (Test Set 1),natural,259,37,38.857143,2.509578,-1.224247,-4.370763,-1.475237,...,-0.762147,-2.312012,0.559362,-5.336716,-1.671388,0.605439,3.162432,-1.077751,2.827472,-0.709370
699,DP30,Validation (Test Set 1),natural,266,38,38.857143,1.057239,-1.448986,-2.316846,-1.961948,...,-1.224898,-1.825636,0.912220,-4.728120,-2.591027,0.710600,3.145586,-1.308840,2.480771,-0.379375


In [18]:
input_df = pregnancy_df.copy()
metabolite_names = input_df.columns[6:]
# df must have feature from column 6 onward and metadata until column 6.
#input_df[metabolite_names] =  input_df[metabolite_names].sub(average_row_mean, axis=0)

input_tensor = torch.tensor(input_df.iloc[:, 6:].values, dtype=torch.float32)
input_tensor[0]

tensor([ 0.6695, -1.4424, -2.2023, -0.6105,  0.5704,  1.4733,  1.6917,  0.6347,
         3.4789,  3.4759,  1.1653, -0.6691,  0.4019, -1.0045,  0.9781, -0.6744,
         0.3813,  2.5527, -0.3893, -2.1971,  0.7286,  1.0932, -4.3114, -1.3493,
         1.6866,  1.1539, -1.9949, -3.2171, -1.5585,  3.3270, -3.1401, -4.0659,
        -0.0598, -2.1093, -2.5716, -1.2221, -7.8121,  0.6287, -0.1930,  2.8206,
        -0.5932, -1.5778, -1.4347, -4.4832, -1.8511,  0.2475, -0.6552, -5.0469,
        -4.3929,  1.2227, -0.5278,  4.5114, -0.5930, -3.3687,  4.2844,  3.0976,
         0.2213,  0.6778, -1.4817, -0.8550,  2.6132, -1.0299,  2.7675, -0.7996,
        -2.0322,  1.4367,  1.1702, -0.3100,  2.8633,  0.2742,  2.2207, -1.6458,
         0.5513,  2.5245,  1.9290,  3.4519, -0.1615,  0.9453,  0.3212,  3.4990,
        -2.9123, -2.2305, -2.5669, -0.0283, -2.3486,  1.6477, -2.0056, -1.2415,
         1.2639,  3.4499, -0.7323,  2.7940,  0.9982,  5.6954, -2.1027,  0.7507,
        -0.9040,  1.6283, -1.8136, -0.76

In [19]:
TestingKoop3

KoopmanModel(
  (embedding): FF_AE(
    (encode): Sequential(
      (0): Linear(in_features=264, out_features=100, bias=True)
      (1): ReLU()
      (2): Linear(in_features=100, out_features=100, bias=True)
      (3): ReLU()
      (4): Linear(in_features=100, out_features=100, bias=True)
      (5): ReLU()
      (6): Linear(in_features=100, out_features=100, bias=True)
      (7): ReLU()
      (8): Linear(in_features=100, out_features=20, bias=True)
    )
    (decode): Sequential(
      (0): Linear(in_features=20, out_features=100, bias=True)
      (1): ReLU()
      (2): Linear(in_features=100, out_features=100, bias=True)
      (3): ReLU()
      (4): Linear(in_features=100, out_features=100, bias=True)
      (5): ReLU()
      (6): Linear(in_features=100, out_features=100, bias=True)
      (7): ReLU()
      (8): Linear(in_features=100, out_features=264, bias=True)
    )
  )
  (operator): LinearizingKoop(
    (linearizer): FFLinearizer(
      (lin_encode): Sequential(
        (0): Linear

In [20]:
predict_fwd

NameError: name 'predict_fwd' is not defined

In [ ]:
import torch.nn as nn

criterion = nn.MSELoss()

criterion(predict_fwd[0], input_tensor[0])

In [ ]:
i = 20
predict_bwd, predict_fwd = TestingKoop3(input_tensor[i], fwd=1)
criterion(predict_fwd[0], input_tensor[i])

In [ ]:
shift_dfs = {}
num_shift=2
for shift in range(num_shift):
    # Perform the Koopman forward prediction
    predict_bwd, predict_fwd = TestingKoop3.predict(input_tensor, fwd=1)

    # Create a DataFrame for this shift
    shift_df = pd.DataFrame(predict_fwd[0].detach().numpy(), columns=metabolite_names)
    shift_df.index = pregnancy_df.index

    shift_df[metabolite_names] =  shift_df[metabolite_names].add(average_row_mean, axis=0)

    metadata_df = pregnancy_df.iloc[:, :6].copy()
    if time_identifier in metadata_df.columns:
        metadata_df[time_identifier] += shift + 1

    shift_df = pd.concat([metadata_df, shift_df], axis=1)

    # Store the DataFrame in the dictionary
    shift_dfs[shift+1] = shift_df
    input_tensor = predict_fwd[0]

In [ ]:
shift_dfs

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

import ipywidgets as widgets



def shift_data(df, num_shift, time_identifier):
    df = df.copy()
    metabolite_names = df.columns[6:]
    # df must have feature from column 6 onward and metadata until column 6.
    df[metabolite_names] =  df[metabolite_names].sub(average_row_mean, axis=0)

    input_tensor = torch.tensor(df.iloc[:, 6:].values, dtype=torch.float32)

    shift_dfs = {}

    for shift in range(num_shift):
        # Perform the Koopman forward prediction
        gnext, x1 = HP_model.koop_forward(input_tensor)

        # Create a DataFrame for this shift
        shift_df = pd.DataFrame(x1.detach().numpy(), columns=metabolite_names)
        shift_df.index = df.index

        shift_df[metabolite_names] =  shift_df[metabolite_names].add(average_row_mean, axis=0)

        metadata_df = df.iloc[:, :6].copy()
        if time_identifier in metadata_df.columns:
            metadata_df[time_identifier] += shift + 1

        shift_df = pd.concat([metadata_df, shift_df], axis=1)

        # Store the DataFrame in the dictionary
        shift_dfs[shift+1] = shift_df

        input_tensor = x1

    return shift_dfs

# Assuming df and interpolated_df are already defined dataframes
def add_trace_and_connect(ax, trace_df, connect_df, df_name, time_identifier, feature_name, time_limit=None, color_values=None):

    if time_limit is not None:
          if isinstance(time_limit, (list, tuple)) and len(time_limit) == 2:
              start_time, end_time = time_limit
              trace_df = trace_df[(trace_df[time_identifier] >= start_time) & (trace_df[time_identifier] <= end_time)]
              connect_df = connect_df[(connect_df[time_identifier] >= start_time) & (connect_df[time_identifier] <= end_time)]

    # Find common indices between trace_df and connect_df
    common_indices = trace_df.index.intersection(connect_df.index)

    # Filter both DataFrames to keep only common indices
    trace_df = trace_df.loc[common_indices]
    connect_df = connect_df.loc[common_indices]

    colors = trace_df[time_identifier].map(color_values)

    # Scatter plot for trace_df
    sc = ax.scatter(
        trace_df[time_identifier],
        trace_df[feature_name],
        s=100,
        c=colors,
        #label=df_name,
        edgecolor='k',
        alpha=0.75
    )

    # Add lines connecting corresponding points
    for i in range(len(connect_df)):
        ax.plot(
            [connect_df[time_identifier].iloc[i], trace_df[time_identifier].iloc[i]],
            [connect_df[feature_name].iloc[i], trace_df[feature_name].iloc[i]],
            color='black',
            linestyle='--',
            linewidth=1,
            alpha=0.5,
            #label='Shifted Data'
        )

    return sc

# Function to plot time series data for a given Subject ID and feature
def plot_feature_timeseries(df, interpolated_df, subject_id, feature_name, time_unit, num_shift, original_point_range):
    # Filter data for the specified subject ID
    if subject_id == 'all':
        subject_data_original = df.copy()
        subject_data_interpolated = interpolated_df.copy()
        birth_ga_weeks = subject_data_interpolated['Birth GA/weeks'].mean()

    else:
        subject_data_original = df[df['Subject ID'] == subject_id]
        subject_data_interpolated = interpolated_df[interpolated_df['Subject ID'] == subject_id]
        birth_ga_weeks = subject_data_interpolated['Birth GA/weeks'].iloc[0]

    feature_average = interpolated_df[feature_name].mean()

    if time_unit == 'Gestational age (GA)/weeks':
        subject_data_original = subject_data_original.sort_values(by='Gestational age (GA)/weeks')
        subject_data_interpolated = subject_data_interpolated.sort_values(by='Gestational age (GA)/weeks')
        x_label = 'Gestational age (GA)/weeks'
    else:
        subject_data_original = subject_data_original.sort_values(by='Gestational age (GA)/days')
        subject_data_interpolated = subject_data_interpolated.sort_values(by='Gestational age (GA)/days')
        x_label = 'Gestational age (GA)/days'

    feature_time_average = interpolated_df.groupby(time_identifier)[feature_name].mean()

    shift_dfs = shift_data(subject_data_interpolated.iloc[:,1:], num_shift, x_label)

    # Plotting
    plt.figure(figsize=(10, 20))

    if subject_id == 'all':
        replicates = sorted(subject_data_original['Subject ID'].unique())
        colormap = plt.cm.tab20
        norm = mcolors.Normalize(vmin=0, vmax=len(replicates))
        color_dict = {replicate: colormap(norm(i)) for i, replicate in enumerate(replicates)}
        for replicate, color in color_dict.items():
            subject_data_original_temp = subject_data_original[subject_data_original['Subject ID'] == replicate]
            subject_data_interpolated_temp = subject_data_interpolated[subject_data_interpolated['Subject ID'] == replicate]
            #plt.plot(subject_data_original_temp[x_label], subject_data_original_temp[feature_name], marker='o', linestyle='-', color=color, label=f'Original Data ({replicate})')
            plt.plot(subject_data_interpolated_temp[x_label], subject_data_interpolated_temp[feature_name], marker='o', linestyle='--', color=color, label=f'Imputed Data ({replicate})')
            #plt.scatter(predicted_yes[x_label], predicted_yes[feature_name], color='r', label='Interpolated Data (Yes)', zorder=5)
    else:
        plt.plot(subject_data_original[x_label], subject_data_original[feature_name], marker='o', linestyle='-', color='g')
        plt.plot(subject_data_interpolated[x_label], subject_data_interpolated[feature_name], marker='o', linestyle='--', color='b')
        #plt.scatter(predicted_yes[x_label], predicted_yes[feature_name], color='r', label='Interpolated Data (Yes)', zorder=5)

    # Draw a red vertical line at the Birth GA/weeks value
    plt.axvline(x=birth_ga_weeks, color='red', linestyle='--', linewidth=2, label='Birth GA/weeks')
    plt.axhline(y=feature_average, color='orange', linestyle='--', linewidth=2, label='Overall Average Value')
    plt.plot(feature_time_average.index, feature_time_average.values, color='cyan', marker='o', linestyle='-', linewidth=3, label='Time Average Value')

    if num_shift > 0:
        connect_df = subject_data_interpolated.copy()

        original_point_indices = connect_df[(connect_df[x_label] >= original_point_range[0]) & (connect_df[x_label] <= original_point_range[1])].index
        # Filter the DataFrame using the indices
        connect_df = connect_df.loc[original_point_indices]

        time_limit = [0, 43]
        colormap = plt.get_cmap('viridis')
        time_values = subject_data_interpolated[time_identifier]
        norm = mcolors.Normalize(vmin=time_values.min(), vmax=time_values.max())
        color_values = {val: colormap(norm(val)) for val in time_values.unique()}

        for shift, shift_df in shift_dfs.items():
            df_name = f'Shifted_{shift}'
            trace_df = shift_df.copy()
            add_trace_and_connect(plt, trace_df, connect_df, df_name, x_label, feature_name, time_limit, color_values)
            connect_df = trace_df
            connect_df = connect_df.loc[original_point_indices]

    plt.title(f'Time Series Plot of {feature_name} for Subject ID {subject_id}')
    plt.xlabel(x_label)
    plt.ylabel(feature_name)
    if time_unit == 'Gestational age (GA)/days':
        plt.gca().xaxis.set_major_locator(plt.MultipleLocator(7))  # Show ticks every 7 days
    if time_unit == 'Gestational age (GA)/weeks':
        plt.gca().xaxis.set_major_locator(plt.MultipleLocator(1))  # Show ticks every week

    plt.grid(True)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)  # Adjust ncol to the number of columns you want in your legend
    plt.tight_layout()
    plt.show()

# Get unique Subject IDs and feature columns
sorted_subject_ids = sorted(plot_df_4['Subject ID'].unique())
sorted_subject_ids.insert(0, 'all')
feature_columns = plot_df_4.select_dtypes(include=[np.float64, np.float32]).columns
time_values = sorted(plot_df_4['Gestational age (GA)/weeks'].unique())

# Create dropdown widgets for Subject ID, Feature, and Time Unit selection
subject_id_dropdown = widgets.Dropdown(
    options=sorted_subject_ids,
    description='Subject ID:',
    disabled=False,
)

feature_dropdown = widgets.Dropdown(
    options=feature_columns,
    description='Feature:',
    disabled=False,
)

time_unit_dropdown = widgets.Dropdown(
    options=['Gestational age (GA)/weeks', 'Gestational age (GA)/days'],
    description='Time Unit:',
    disabled=False,
)

shift_dropdown = widgets.Dropdown(
    options=np.arange(0, 40),
    description='Shift:',
    disabled=False,
)


# Function to create interactive widgets
def plot_interactive_timeseries_plot(subject_id, feature_name, time_unit, num_shift):
    time_slider = widgets.IntRangeSlider(
            value=[0, 43],
            min=0,
            max=43,
            step=1,
            description='Original Timepoints:',
            continuous_update=True,
            orientation='horizontal',
            readout=True,
            readout_format='d',
        )


    # Define interactive plot function
    def interactive_plot(subject_id, feature_name, time_unit, num_shift, time_slider_value):
        plot_feature_timeseries(plot_df_3, plot_df_4, subject_id, feature_name, time_unit, num_shift, time_slider_value)

    # Create interactive widgets
    ui = widgets.VBox([subject_id_dropdown, feature_dropdown, time_unit_dropdown, shift_dropdown, time_slider])
    out = widgets.interactive_output(interactive_plot,
                                     {'subject_id': subject_id_dropdown,
                                      'feature_name': feature_dropdown,
                                      'time_unit': time_unit_dropdown,
                                      'num_shift': shift_dropdown,
                                      'time_slider_value': time_slider})

    display(ui, out)

# Display the interactive widget
plot_interactive_timeseries_plot(subject_id_dropdown.value,
                                 feature_dropdown.value,
                                 time_unit_dropdown.value,
                                 shift_dropdown.value)

